In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from src import hrNet
from src import readFrames

In [ ]:
patch_tmin1, patch_t, patch_tplus1 = readFrames("KristenAndSara_10frames_512x256.yuv420p",[512,256],1,7)

In [ ]:
patch_tSmall = patch_t[:192,:192,:]
plt.imshow(patch_tSmall/255.0)

In [ ]:
inputPatches = np.concatenate((patch_tSmall,patch_tSmall,patch_tSmall),axis=-1)
inputPatches = np.expand_dims(inputPatches,axis=0)

In [ ]:
hrNetModel = hrNet(2, [32, 64, 128, 256], 5).model()

In [ ]:
outputPatch = hrNetModel(inputPatches,training=False)
plt.imshow(outputPatch[0]/255.0)

In [ ]:
import lpips
import torch
from src import image_preprocess

In [ ]:
class lpipsClass():
    def __init__(self):
        self.loss_fn_vgg = lpips.LPIPS(net='vgg') # closer to "traditional" perceptual loss, when used for optimization

    def lpipsLossFunc(self, x, y):
        x, y = torch.tensor(x), torch.tensor(y)
        d = self.loss_fn_vgg(x, y)
        return d

    def computeLpipsFrame(self,restored_frame : tf.Tensor,original_frame : tf.Tensor):
        img0 = restored_frame
        img0 = image_preprocess(img0)
        img0 = tf.transpose(img0,perm=[0,3,1,2])
        #img0 = np.expand_dims(img0,axis=0)
        #img0 = np.moveaxis(img0,-1,1)
        #img0 = np.float32(img0)
        img0 = img0.numpy()
        # the restored frame
        img1 = original_frame
        img1 = image_preprocess(original_frame)
        img1 = tf.transpose(img1,perm=[0,3,1,2])
        img1 = img1.numpy()
        #img1 = np.expand_dims(img1,axis=0)
        #img1 = np.moveaxis(img1,-1,1)
        #img1 = np.float32(img1)
        # Converting from numpy to pytorch
        img0 = torch.tensor(img0)
        img1 = torch.tensor(img1)

        return self.lpipsLossFunc(img0,img1)

In [ ]:
lpipsInst = lpipsClass()

In [ ]:
hrNetModel.compile(run_eagerly=True,optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),loss=lpipsInst.computeLpipsFrame)

In [ ]:
hrNetModel.fit(inputPatches[:,:,:,:],inputPatches[:,:,:,:3],epochs=1)

In [ ]:
from subprocess import run

In [ ]:
run("ffmpeg -version",capture_output=True, shell=True).stdout.decode("utf-8").rstrip("\n")

In [1]:
from library import returnMetric
import numpy as np 

cambi = returnMetric(np.random.uniform(0, 1, (1, 192, 192, 3)), np.random.uniform(0, 1, (1, 192, 192, 3)))

2023-02-22 15:41:43.509725: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
problem loading feature extractor: cambi


vmaf -r /home/nwaliv/MAIProject-main/processing/patchesRef.yuv -d /home/nwaliv/MAIProject-main/processing/patchesDeg.yuv -w 192 -h 192 -p 420 -b 8 --csv -o /home/nwaliv/MAIProject-main/processing/scores.csv --feature cambi


FileNotFoundError: [Errno 2] No such file or directory: '/home/nwaliv/MAIProject-main/processing/scores.csv'